# Implementação do algortimo completo, sem compromisso com  perfomance

## Carregando as matrizes de dissimilaridade

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import numpy as np

DATA_BASE_PATH = "/content/drive/My Drive/2020.1/AM/Fuzzy Clustering/data"

FAC_FILE = os.path.join(DATA_BASE_PATH, "mfeat-fac-dissimilarity.npy")
FOU_FILE = os.path.join(DATA_BASE_PATH, "mfeat-fou-dissimilarity.npy")
KAR_FILE = os.path.join(DATA_BASE_PATH, "mfeat-kar-dissimilarity.npy")

fac_dis = np.load(FAC_FILE)
fou_dis = np.load(FOU_FILE)
kar_dis = np.load(KAR_FILE)


## Definindo a função match

In [ ]:

# def cluster_matching_function(weight_matrix, 
#                               cluster_qtd,
#                               element, 
#                               prototypes, 
#                               dissimilarity_matrices):
#   k = cluster_qtd
#   p = len(dissimilarity_matrices)
#   Gk = prototypes[k]
#   D = 0

#   for j in range(p):
#     dj = dissimilarity_matrices[j]
#     inner_sum = sum([dj[element, e] for e in Gk])
#     D += weight_matrix[k,j] * inner_sum
  
#   return D

def cluster_matching_function(weight_matrix, 
                              cluster_number,
                              element, 
                              prototypes, 
                              dissimilarity_matrices):
  k = cluster_number
  D = dissimilarity_matrices
  p = len(D)
  Gk = prototypes[k]

  weights = weight_matrix[k,:p]
  dissimilarities_sum = np.array([sum([dj[element, e] for e in Gk]) for dj in D])
  return np.dot(weights, dissimilarities_sum)



## Definindo a função objetivo

In [ ]:
import functools
import operator
import math 

# def objective_function(clusters_qtd, 
#                        elements_qtd, 
#                        adequacy_criterion, 
#                        m, 
#                        weight_matrix, 
#                        prototypes,
#                        dissimilarity_matrices):
#   J = 0
#   u = adequacy_criterion
#   m_func = cluster_matching_function

#   for k in clusters_qtd:
#     D = 0

#     for i in range(elements_qtd):
#       D+= (u[i,k] ** m) * m_func(weight_matrix,
#                                   clusters_qtd,
#                                   i,
#                                   prototypes,
#                                   dissimilarity_matrices)
#     J += D

#   return J

def objective_function(clusters_qtd, 
                       elements_qtd, 
                       adequacy_criterion, 
                       m, 
                       weight_matrix, 
                       prototypes,
                       dissimilarity_matrices):
  
  u = np.power(adequacy_criterion, m)
  l = weight_matrix
  D = dissimilarity_matrices
  K = clusters_qtd
  Gk = prototypes
  n = elements_qtd
  match = cluster_matching_function

  J = [sum([u[i,k] * match(l, k, i, Gk, D) for i in range(n)]) 
          for k in range(K)]

 
  return sum(J)
    

## Definindo a função da cálculo de protótipos

In [ ]:
def get_prototypes(elements_qtd, 
                  q, 
                  m, 
                  s,
                  cluster_number,
                  adequacy_criterion,
                  dissimilarity_matrices, 
                  weight_matrix):
  G = []
  k = cluster_number,
  D = dissimilarity_matrices
  u = adequacy_criterion
  l = weight_matrix
  n = elements_qtd
  p = len(D)

  def dist(element):
    return element, sum([(u[i,k]**m) * sum([(l[j,k])**s * D[j][element,i] for j in range(p)])
          for i in range(elements_qtd)
          
          ]
        )
    

  while (len(G) < q):
    all_distances = (dist(i) for i in range(n) if i not in G)
    G.append(min(all_distances, key=functools.itemgetter(1)))

  return G

## Definindo a função de computo da matriz de relevâcia

In [ ]:
def compute_relevance_weights(clusters_qtd,
                              dissimilarity_matrices,
                              prototypes,
                              elements_qtd,
                              adequacy_criterion,
                              m):
  
    D = dissimilarity_matrices
    p = len(D)
    Gk = prototypes
    K = clusters_qtd
    n = elements_qtd
    u = np.power(adequacy_criterion, m)
    match = cluster_matching_function
    l = np.zeros(K, p)

    for k in range(K):
      for j in range(p):
        weight_dissimilarity_sum = [sum([u[i,k] * match(l, k, i, Gk, D[h]) for i in range(n)])
                                    for h in range(p)]
                                    
        weight_dissimilarity_prod = functools.reduce(operator.mul, weight_dissimilarity_sum)
        weight_dissimilarity_sum = sum([u[i,k] * match(l, k, i, Gk, D[j]) for i in range(n)])
        l[k,j] = math.pow(weight_dissimilarity_prod, 1/p) / weight_dissimilarity_sum

    return l